In [1]:
import random
import string

In [2]:
env= 'cap1234'
min_len = 3
max_len = 10
#환경정보는 3~10 자리수로 구성

def generate_word(length):
    result = ''
    word = ''.join(random.sample(string.ascii_letters + string.digits, k=length))
    return word
# 문자 + 숫자로 이루어진 length 길이의 단어열을 생성

def generate_population(size, min_len, max_len):
    population = [] #세대 배열
    for i in range(size):
        length = i % (max_len - min_len + 1) + min_len #균등하게 mix_len ~ max_len 길이로 지정
        population.append(generate_word(length))
    return population
#생성시킬 1세대 유전자 개수, 문자열의 최소길이, 문자열의 최대길이 만큼 출력. 문자열의 길이는 균등하게 부여됨

pop = generate_population(size=100, min_len=min_len, max_len=max_len)

In [3]:
def fit_test(env,gen):
    score = 0
    if(len(env) != len(gen)):
        return score
    len_score = 1
    score += len_score

    for i in range(len(env)):
        if env[i] == gen[i]:
            score += 1

    return score / (len(env) + len_score) * 100

In [4]:
def compute_performace(population, password):
    performance_list = []
    for individual in population:
        score = fit_test(password, individual)

        # we can predict length of password
        if score > 0:
            pred_len = len(individual)
        performance_list.append([individual, score])

    population_sorted = sorted(performance_list, key=lambda x: x[1], reverse=True)
    return population_sorted, pred_len

def select_survivors(population_sorted, best_sample, lucky_few, password_len):
    next_generation = []

    for i in range(best_sample):
        if population_sorted[i][1] > 0:
            next_generation.append(population_sorted[i][0])

    lucky_survivors = random.sample(population_sorted, k=lucky_few)
    for l in lucky_survivors:
        next_generation.append(l[0])
    
    # generate new population if next_generation is too small
    while len(next_generation) < best_sample + lucky_few:
        next_generation.append(generate_word(length=password_len))

    random.shuffle(next_generation)
    return next_generation

pop_sorted, pred_len = compute_performace(pop, env)
survivors = select_survivors(pop_sorted, best_sample=20, lucky_few=20, password_len=pred_len)

In [5]:
def create_child(individual1, individual2):
    child = ''
    min_len_ind = min(len(individual1), len(individual2))
    for i in range(min_len_ind):
        if (int(100 * random.random()) < 50):
            child += individual1[i]
        else:
            child += individual2[i]
    return child

# 우리는 그렇게 자비롭지 않기때문에 부모를 내 맘대로 짝지어줄 것이다
def create_children(parents, n_child):
    next_population = []
    for i in range(int(len(parents)/2)):
        for j in range(n_child):
            next_population.append(create_child(parents[i], parents[len(parents) - 1 - i]))
    return next_population

children = create_children(survivors, 5)

In [6]:
def mutate_word(word):
    idx = int(random.random() * len(word))
    if (idx == 0):
        word = random.choice(string.ascii_letters + string.digits) + word[1:]
    else:
        word = word[:idx] + random.choice(string.ascii_letters + string.digits) + word[idx+1:]
    return word

def mutate_population(population, chance_of_mutation):
    for i in range(len(population)):
        if random.random() * 100 < chance_of_mutation:
            population[i] = mutate_word(population[i])
    return population

new_generation = mutate_population(population=children, chance_of_mutation=10)

In [7]:
def generate_env():
    result = ''
    word = ''.join(random.sample(string.ascii_letters + string.digits, k=10))
    env = word
    return env

def random_child():
    n_child = random.randint(1, 7)
    return n_child

def random_luck():
    lucky_few = random.randint(1,10)
    return lucky_few

In [8]:
env = generate_env()
n_generation = 10000 #생성시킬 세대 수
population = 100 #생성시킬 1세대 유전자 수
best_sample = 30 #적합하여 살아남은 유전자의 수
lucky_few = 10 #운좋게 살아남은 놈들
n_child = 5 #낳을 자식의 수
chance_of_mutation = 10 #돌연변이율(%)
cnt = 0

pop = generate_population(size=population, min_len=min_len, max_len=max_len)

for g in range(n_generation):
    if g==0:
        print(env)
    pop_sorted, pred_len = compute_performace(population=pop, password=env)
    if g !=0 and g%100 ==0: # ~세대 마다 일어나는 변화
        env = generate_env()
        print()
        print('Environment change! %s' % env)
        cnt =cnt+1
        print()
    #환경 변화 
    
    if int(pop_sorted[0][1]) == 100:
        gen = g-cnt*100
        print('완벽히 적응하게된차수는', gen,'세대,','유전자는', (pop_sorted[0][0]), '입니다')
        print('그동안 환경변화는',cnt,'번 일어났습니다')
        break
    lucky_few = random_luck()
    
    survivors = select_survivors(population_sorted=pop_sorted, best_sample=best_sample, lucky_few=lucky_few, password_len=pred_len)
    
    children = create_children(parents=survivors, n_child=n_child)

    new_generation = mutate_population(population=children, chance_of_mutation=10)
    
    pop = new_generation
    # 세대출력
    print('===== %sth Generation =====' % (g + 1))
    print(pop_sorted[0])

1d4UpcGxki
===== 1th Generation =====
['trd9TnGPAM', 18.181818181818183]
===== 2th Generation =====
['WrgECjh1kI', 18.181818181818183]
===== 3th Generation =====
['Aeis93Gyls', 18.181818181818183]
===== 4th Generation =====
['grHGCaGfks', 27.27272727272727]
===== 5th Generation =====
['WEM9jjG1ks', 27.27272727272727]
===== 6th Generation =====
['gdvG9jGtkx', 36.36363636363637]
===== 7th Generation =====
['9d49wLG0Ax', 36.36363636363637]
===== 8th Generation =====
['9d49TLG0ks', 45.45454545454545]
===== 9th Generation =====
['9d4GCaG0ks', 45.45454545454545]
===== 10th Generation =====
['9d4GCaG0ks', 45.45454545454545]
===== 11th Generation =====
['gd47PLG0kx', 45.45454545454545]
===== 12th Generation =====
['gd49jdGfkI', 45.45454545454545]
===== 13th Generation =====
['gd49TaGfkx', 45.45454545454545]
===== 14th Generation =====
['gd47TaG0kx', 45.45454545454545]
===== 15th Generation =====
['gd49CLGtkT', 45.45454545454545]
===== 16th Generation =====
['9d49CdGfkx', 45.45454545454545]
===

===== 186th Generation =====
['uD4pTIw6k2', 63.63636363636363]
===== 187th Generation =====
['uD4pTIw6ki', 63.63636363636363]
===== 188th Generation =====
['uD4pTIw6k2', 63.63636363636363]
===== 189th Generation =====
['uD4pTIw6ki', 63.63636363636363]
===== 190th Generation =====
['uD4pTIw6ki', 63.63636363636363]
===== 191th Generation =====
['uDLpTIw6ki', 63.63636363636363]
===== 192th Generation =====
['uD4ZTIw6k2', 72.72727272727273]
===== 193th Generation =====
['uD4ZTIw6k2', 72.72727272727273]
===== 194th Generation =====
['uD4ZTIw6k2', 72.72727272727273]
===== 195th Generation =====
['uD4ZTIw6ki', 72.72727272727273]
===== 196th Generation =====
['uD4ZTIw6ki', 72.72727272727273]
===== 197th Generation =====
['uD4ZTIw6k2', 72.72727272727273]
===== 198th Generation =====
['uD4ZTIw6ki', 72.72727272727273]
===== 199th Generation =====
['uD4ZTIw6ki', 72.72727272727273]
===== 200th Generation =====
['uD4ZTIw6k2', 72.72727272727273]

Environment change! QkIq01dSh5

===== 201th Generation

['JKIV21EY19', 36.36363636363637]
===== 456th Generation =====
['JKIV2jEY19', 36.36363636363637]
===== 457th Generation =====
['JKIV2jEY19', 36.36363636363637]
===== 458th Generation =====
['qlIV2jEY19', 36.36363636363637]
===== 459th Generation =====
['3KIV21EY19', 45.45454545454545]
===== 460th Generation =====
['3KIV21Ej19', 45.45454545454545]
===== 461th Generation =====
['3KIV2jEj1T', 54.54545454545454]
===== 462th Generation =====
['3KRV21Ej1T', 54.54545454545454]
===== 463th Generation =====
['3KIV2jEj1T', 54.54545454545454]
===== 464th Generation =====
['3KIV2jEj1T', 54.54545454545454]
===== 465th Generation =====
['3KIV2TEj1T', 54.54545454545454]
===== 466th Generation =====
['3KIV21EY1T', 54.54545454545454]
===== 467th Generation =====
['3KIV2jEY1T', 54.54545454545454]
===== 468th Generation =====
['3KIV2TEh1T', 54.54545454545454]
===== 469th Generation =====
['3KIV21EYmT', 54.54545454545454]
===== 470th Generation =====
['3KIV2jEh1T', 54.54545454545454]
===== 471th Generatio

===== 680th Generation =====
['R6lIMcWtGB', 72.72727272727273]
===== 681th Generation =====
['f6lIMcWtGM', 72.72727272727273]
===== 682th Generation =====
['k6lIMcWtGB', 72.72727272727273]
===== 683th Generation =====
['k6lIMcWtGB', 72.72727272727273]
===== 684th Generation =====
['k6lIMcWtGB', 72.72727272727273]
===== 685th Generation =====
['k6lIMcWtGB', 72.72727272727273]
===== 686th Generation =====
['k6lIMcWtGB', 72.72727272727273]
===== 687th Generation =====
['k6lIMcWtGB', 72.72727272727273]
===== 688th Generation =====
['f6lIMcWtGB', 72.72727272727273]
===== 689th Generation =====
['f6lIMcWtGB', 72.72727272727273]
===== 690th Generation =====
['f6lIMcWtGB', 72.72727272727273]
===== 691th Generation =====
['f6lIMcWtGB', 72.72727272727273]
===== 692th Generation =====
['f6lIMcWtGB', 72.72727272727273]
===== 693th Generation =====
['k6lIMcWtGB', 72.72727272727273]
===== 694th Generation =====
['f6lIMcWtGo', 72.72727272727273]
===== 695th Generation =====
['f6lIMcWtGB', 72.72727272

===== 920th Generation =====
['ycrKlKzQEJ', 27.27272727272727]
===== 921th Generation =====
['ycrKlKzQEJ', 27.27272727272727]
===== 922th Generation =====
['ycrKlTzQEJ', 27.27272727272727]
===== 923th Generation =====
['ycrKlKzQEJ', 27.27272727272727]
===== 924th Generation =====
['ycrKlKzQEJ', 27.27272727272727]
===== 925th Generation =====
['ycrKlKzQEJ', 27.27272727272727]
===== 926th Generation =====
['EcrKlKzQhJ', 27.27272727272727]
===== 927th Generation =====
['EcrKlKzQEJ', 27.27272727272727]
===== 928th Generation =====
['EcrzlAzQhJ', 27.27272727272727]
===== 929th Generation =====
['ycrzlKzQEJ', 27.27272727272727]
===== 930th Generation =====
['ycrKlK88EJ', 27.27272727272727]
===== 931th Generation =====
['ycrKlKzQKJ', 27.27272727272727]
===== 932th Generation =====
['ycrKlAz6EJ', 27.27272727272727]
===== 933th Generation =====
['ycrKlKz6Ex', 27.27272727272727]
===== 934th Generation =====
['ycrKlKz80J', 27.27272727272727]
===== 935th Generation =====
['ycrKlKz6EJ', 27.27272727

...

['qE6HKpHLQs', 63.63636363636363]
===== 3190th Generation =====
['qE6HKpHLQs', 63.63636363636363]
===== 3191th Generation =====
['qE6HKpELQs', 63.63636363636363]
===== 3192th Generation =====
['qE6HEpHOQs', 63.63636363636363]
===== 3193th Generation =====
['qE6HEpEOQs', 63.63636363636363]
===== 3194th Generation =====
['qE6HEpHZQs', 63.63636363636363]
===== 3195th Generation =====
['qE6HEpEOQs', 63.63636363636363]
===== 3196th Generation =====
['qE6H6pHLQs', 63.63636363636363]
===== 3197th Generation =====
['qE6HEpHZQs', 63.63636363636363]
===== 3198th Generation =====
['qE6HEpHZQs', 63.63636363636363]
===== 3199th Generation =====
['qE6H6pELQs', 63.63636363636363]
===== 3200th Generation =====
['qE6HEpHLQs', 63.63636363636363]

Environment change! dIUr8Vy2u4

===== 3201th Generation =====
['qE6HEpHLQs', 63.63636363636363]
===== 3202th Generation =====
['qE6H6pHLQs', 9.090909090909092]
===== 3203th Generation =====
['qE6H6pHZQi', 9.090909090909092]
===== 3204th Generation =====
['qE6HE

['K1b8UHy2Zo', 36.36363636363637]
===== 3437th Generation =====
['K3b8UHu2Zo', 36.36363636363637]
===== 3438th Generation =====
['K3b8UPy2Zo', 36.36363636363637]
===== 3439th Generation =====
['K3b8UHu2Zo', 36.36363636363637]
===== 3440th Generation =====
['K3b8U7s2Zo', 45.45454545454545]
===== 3441th Generation =====
['K1b8UHs2Zo', 45.45454545454545]
===== 3442th Generation =====
['K3b8UHyaZo', 45.45454545454545]
===== 3443th Generation =====
['K1b8UHsaZo', 54.54545454545454]
===== 3444th Generation =====
['K1b8UHsaZo', 54.54545454545454]
===== 3445th Generation =====
['K1b8UHsaZo', 54.54545454545454]
===== 3446th Generation =====
['K1b8UHsaZo', 54.54545454545454]
===== 3447th Generation =====
['K3b8UHsaZo', 54.54545454545454]
===== 3448th Generation =====
['K1b8UHsaZo', 54.54545454545454]
===== 3449th Generation =====
['K1i8UHsaZo', 54.54545454545454]
===== 3450th Generation =====
['K1b8UHsaZo', 54.54545454545454]
===== 3451th Generation =====
['K1b8UHsaZo', 54.54545454545454]
===== 

===== 3684th Generation =====
['F423bxssBI', 72.72727272727273]
===== 3685th Generation =====
['Fn23bxssBI', 72.72727272727273]
===== 3686th Generation =====
['FR23bxssBI', 72.72727272727273]
===== 3687th Generation =====
['Fn23bxssBI', 72.72727272727273]
===== 3688th Generation =====
['F423bxssBI', 72.72727272727273]
===== 3689th Generation =====
['FR23bxmsBI', 72.72727272727273]
===== 3690th Generation =====
['FR23bxssBI', 72.72727272727273]
===== 3691th Generation =====
['FR23bxmsBI', 72.72727272727273]
===== 3692th Generation =====
['FR23bxmsBI', 72.72727272727273]
===== 3693th Generation =====
['FR23bxmsBI', 72.72727272727273]
===== 3694th Generation =====
['Fl23bxmsBI', 72.72727272727273]
===== 3695th Generation =====
['Fi237xmsBI', 72.72727272727273]
===== 3696th Generation =====
['Fl237xssBI', 72.72727272727273]
===== 3697th Generation =====
['Fi237xmsBI', 72.72727272727273]
===== 3698th Generation =====
['FR23bxmsBI', 72.72727272727273]
===== 3699th Generation =====
['FR237xms

===== 3931th Generation =====
['FK3S1GlWLS', 63.63636363636363]
===== 3932th Generation =====
['FK3S1IlWLS', 63.63636363636363]
===== 3933th Generation =====
['FK3S1IlWLS', 63.63636363636363]
===== 3934th Generation =====
['FK3S1GlWLS', 63.63636363636363]
===== 3935th Generation =====
['FK3S1GlWLS', 63.63636363636363]
===== 3936th Generation =====
['FK3SEGlWLS', 63.63636363636363]
===== 3937th Generation =====
['FK3S1GlWLS', 63.63636363636363]
===== 3938th Generation =====
['FK3SNGlWLS', 63.63636363636363]
===== 3939th Generation =====
['FK3S1IlWLS', 63.63636363636363]
===== 3940th Generation =====
['FK3S1IlWLS', 63.63636363636363]
===== 3941th Generation =====
['FK3S1IwWLS', 63.63636363636363]
===== 3942th Generation =====
['FK3S1GwWLS', 63.63636363636363]
===== 3943th Generation =====
['FK3S1IwWLS', 63.63636363636363]
===== 3944th Generation =====
['FK3S1IwWLS', 63.63636363636363]
===== 3945th Generation =====
['FK3S1IwWLS', 63.63636363636363]
===== 3946th Generation =====
['FK3S1IwW

['DTlfribESB', 81.81818181818183]
===== 4179th Generation =====
['DTlfribESB', 81.81818181818183]
===== 4180th Generation =====
['DTlfribESB', 81.81818181818183]
===== 4181th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4182th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4183th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4184th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4185th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4186th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4187th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4188th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4189th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4190th Generation =====
['0TlfribESB', 90.9090909090909]
===== 4191th Generation =====
['0TlfribESB', 90.9090909090909]
완벽히 적응하게된차수는 91 세대, 유전자는 0TtfribESB 입니다
그동안 환경변화는 41 번 일어났습니다
